### Route Variance Identification Over Time.

Recent observations shows small chages in routes over time. Specifically in the following fields:
* route ID
* route short name
* route long name
* route desc

Need to observe these route changes in order to account for these changes in future analyses.

### Objective
1. Query data from `fct_monthly_routes` to help identify variences in Routes. Query for 2023, a couple of months. 
2. Save data to GCS `gtfs_schedule` bucket
3. Filter down data to `Sacramento Regional Transit`, identify and observe routes for any variences


### Last Update 12/16/2023

Started off by looking into `dim_routes` table. Decided that `dim_routes` didnt have enough data so moved to looking into `fct_monthyl_routes` and `helpers.import_scheduled_trips`. Both tables were filtered to 6 months (April 2023 to September 2023) and to the Sac RT routes. `dim_providers_gtfs_data` was also identified as a possible join table in order to tie together feedkey, gtfs dataset key and source record id.


However, when analyzing the data from `helpers.import_scheduled_trips`, No data was found for June 2023 and 2 gtfs schedule dataset keys were found. In contrast, the `fct-monthly_route` & `dim_providers_gtfs_data` tables had 1 gtfs schedule dataset key. Further analysis is needed to explain.


Currently the 2 df I have now are:
1. `fct_monthly_routes/dim_providers_gtfs_data` (aka sac_m4m9)
2. `helpers.import_scheduled_trips` (aka sac_trip_desc)

Latest steps i did. Cleaning up the `sac_m4m9 df`, then tested merging both df together (knowing that `sac_trip_desc` had multiple gtfs dataset keys) to see what kind of df is produced.



In [1]:
#imports copied from download_vehicle_position.py script

import datetime
import gcsfs
import geopandas as gpd
import pandas as pd
import shapely
import sys

from calitp_data_analysis.tables import tbls
from calitp_data_analysis import utils
from loguru import logger
from siuba import *

from shared_utils import schedule_rt_utils
from shared_utils import geography_utils

#rt_segment_speeds/segment_speed_utils
from segment_speed_utils import helpers

#rt_segment_speeds/segment_speed_utils/project_vars.py
from segment_speed_utils.project_vars import SCHED_GCS

#_shared_utils/shared_utils/rt_dates.py
from shared_utils import rt_dates

ModuleNotFoundError: No module named 'shared_utils'

In [ ]:
## test to query fct_monthly_routes. DO NOT RUN

#def get_monthly_routes(
#        year: str,
#        months: list
#    ) -> pd.DataFrame:    
#    
#        df = (tbls.mart_gtfs.fct_monthly_routes()
#              >> filter(_.year == year)
#              >> filter(_.month.isin(months))
#              >> select(_.key, _.source_record_id,
#                        _.name,
#                        _.route_id, _.shape_id,
#                        _.month,
#                        _.year,
#                       _.pt_array)
#                  >> collect()
#             )
#        return df

In [ ]:
# df_pt = get_monthly_routes(2023, [4, 5, 6, 7, 8, 9])

In [ ]:
#from metabase, joined fct_monthly_routes with dim_providers_gtfs_data.saved results to gcs
#test reading in data for  sacramento schedule m4 to m9,

#df = pd.read_csv('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sacrament_schedule_route_id_2023_m4_m9.csv')

In [ ]:
#df.shape

## cleaning metabase data of fct_monthly_routes and dim_providers_gtfs_data

In [ ]:
#looking at columns name
#df.columns

#function that replaces spaces with _ and make lower case
#effectively making snakecase
def cleaner(df):
    df.columns = df.columns.str.replace(' ','_')
    df.columns = df.columns.str.lower()
    return df

In [ ]:
#cleaner(df)

In [ ]:
#removing dim providers string
#df.columns = df.columns.str.replace('dim_provider_gtfs_data_→_','')


In [ ]:
#df.columns

#need a way to get point array back
#merge df with regular fct_monthly_routes on key

merge = pd.merge(df, df_pt, on=['key', 
                                'name', 
                                'month', 
                                'source_record_id', 
                                'route_id',
                                'year',
                               ])

display(merge.shape)
display(type(merge))
list(merge.columns)


In [ ]:
#merge.head(3)

merge2=merge.drop(columns=['distinct_values_of_key',
 'shape_id_y'])
merge2.columns

In [ ]:
#makeing merge into a gdf now that i have pt geom back

#sac_m4m9= geography_utils.make_routes_gdf(merge2, 'EPSG:4326')

type(sac_m4m9)
display(sac_m4m9.geometry.name)
display(list(sac_m4m9.columns))

In [ ]:
#sac_m4m9.plot()

In [ ]:
#uploading as geoparquet to GCS
#utils.geoparquet_gcs_export(sac_m4m9, "gs://calitp-analytics-data/data-analyses/gtfs_schedule/", "sac_route_variance_m4m9_geo")

## start here

In [2]:
#attempt at reading in geoparquet FROM GCS
#fct_monthly_routes/dim_providers_gtfs_data
sac_m4m9 = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_route_variance_m4m9_geo.parquet')

In [3]:
#attempt to read in dataframe from approach 2
#helpers.import_scheduled_trips
sac_trip_desc = pd.read_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_trips_route_identification_2023_m04_m09.parquet')

## Dataframe Comparison
Comparing the data within the `fct_monthly_routes/dim_providers_gtfs_data` (aka sac_m4m9) and the `helpers.import_scheduled_trips` (aka sac_trip_desc)

In [4]:
#info about fct_monthly_routes/dim_providers_gtfs_data df
display(type(sac_m4m9))
display(sac_m4m9.shape)
display(sac_m4m9.head(3))

geopandas.geodataframe.GeoDataFrame

(1179, 18)

,key,schedule_gtfs_dataset_key,name,month,year,month_last_day,valid_from,valid_to,source_record_id,route_id,shape_id_x,organization_name,service_name,schedule_gtfs_dataset_name,organization_source_record_id,service_source_record_id,schedule_source_record_id,geometry
0,c5f18e65ac0287bb0fd02f379ca29ea6,cb3074eb8b423dfc5acfeeb0de95eb82,Sacramento Schedule,4,2023,2023-04-30,2023-10-25T00:00:00-07:00,2098-12-31T00:00:00-08:00,recbzZQUIdMmFvm1r,001,45267,Sacramento Regional Transit District,SacRT Light Rail,Sacramento Schedule,recX9lccSE1jmjsmG,rec4ngaW4prJCvuQv,recbzZQUIdMmFvm1r,"LINESTRING (-121.26715 38.67928, -121.26715 38..."
1,c5f18e65ac0287bb0fd02f379ca29ea6,cb3074eb8b423dfc5acfeeb0de95eb82,Sacramento Schedule,4,2023,2023-04-30,2023-10-25T00:00:00-07:00,2098-12-31T00:00:00-08:00,recbzZQUIdMmFvm1r,001,45267,Sacramento Regional Transit District,Sacramento Regional Transit District Bus,Sacramento Schedule,recX9lccSE1jmjsmG,recucpbla6cP2ZdsV,recbzZQUIdMmFvm1r,"LINESTRING (-121.26715 38.67928, -121.26715 38..."
2,c5f18e65ac0287bb0fd02f379ca29ea6,cb3074eb8b423dfc5acfeeb0de95eb82,Sacramento Schedule,4,2023,2023-04-30,2023-10-30T00:00:00-07:00,2098-12-31T00:00:00-08:00,recbzZQUIdMmFvm1r,001,45267,City of Rancho Cordova,Rancho CordoVan,Sacramento Schedule,rec43oyrfhtPDdRHj,recthtqJiccuHePry,recbzZQUIdMmFvm1r,"LINESTRING (-121.26715 38.67928, -121.26715 38..."


In [24]:
sac_m4m9.month.value_counts()

8    201
9    198
4    195
5    195
6    195
7    195
Name: month, dtype: int64

In [25]:
#need to update months column to say month name instead of number
replace_month= {
    4:'april',
    5:'may',
    6:'june',
    7:'july',
    8:'aug',
    9:'sep'
}

In [26]:
sac_m4m9['month'] = sac_m4m9['month'].replace(replace_month)


In [27]:
sac_m4m9.month.value_counts()

aug      201
sep      198
april    195
may      195
june     195
july     195
Name: month, dtype: int64

In [5]:
#info on helpers.import_scheduled_trips df
display(type(sac_trip_desc))
display(sac_trip_desc.shape)
display(sac_trip_desc.head(3))

pandas.core.frame.DataFrame

(326, 8)

,feed_key,name,schedule_gtfs_dataset_key,route_id,route_short_name,route_long_name,route_desc,month
0,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,001,1,GREENBACK,None,april
1,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,011,11,NATOMAS/LAND PARK,None,april
2,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,013,13,NATOMAS/ARDEN,None,april


In [23]:
#no data for June 2023?
sac_trip_desc.month.value_counts()

sep      66
april    65
may      65
july     65
aug      65
Name: month, dtype: int64

In [6]:
#comparing gtfs dataset keys
display(sac_m4m9.schedule_gtfs_dataset_key.value_counts())
display(sac_trip_desc.schedule_gtfs_dataset_key.value_counts())

#observed 1 dataset key in `sac_m4m9`, but 2 dataset keys in `sac_trip_desc`
#`sac_trip_desc` pulls data from `helpers.import_scheduled_trips`


cb3074eb8b423dfc5acfeeb0de95eb82    1179
Name: schedule_gtfs_dataset_key, dtype: int64

43a1e46d592a1ee647bce8422c68460c    260
cb3074eb8b423dfc5acfeeb0de95eb82     66
Name: schedule_gtfs_dataset_key, dtype: int64

In [29]:
#try to merge on name
# and another join on gtfs dataset key
test = sac_m4m9.merge(sac_trip_desc, 
                      how='outer', 
                      on=['name','schedule_gtfs_dataset_key','route_id','month'], 
                      indicator=True)


In [30]:
test.shape


(1439, 23)

In [31]:
list(test.columns)

['key',
 'schedule_gtfs_dataset_key',
 'name',
 'month',
 'year',
 'month_last_day',
 'valid_from',
 'valid_to',
 'source_record_id',
 'route_id',
 'shape_id_x',
 'organization_name',
 'service_name',
 'schedule_gtfs_dataset_name',
 'organization_source_record_id',
 'service_source_record_id',
 'schedule_source_record_id',
 'geometry',
 'feed_key',
 'route_short_name',
 'route_long_name',
 'route_desc',
 '_merge']

clean up the values, especially in month (change numbers to str names)
use the merge to identify whats the same/not the same in each df



In [ ]:
for i in string_cols:        
    gdf[i] = gdf[i].str.title().str.lstrip().str.rstrip()        
    gdf[i] = gdf[i].replace(r'\s+', ' ', regex=True)

In [ ]:
df.select_dtypes(include=['object'])

In [ ]:
#comparing months
display(sac_m4m9.month.value_counts())
display(sac_trip_desc.month.value_counts())
#missing June 2023 from sac_trip_desc??
#so for the same time period, sac_m4m9 had all the same dataset key, but sac_trip_desc has different keys. 
#again, confirming why gtfs dataset key is not the most stable over time


In [ ]:
sac_trip_desc.pivot_table(index = ['schedule_gtfs_dataset_key','month'],
                      values = ['route_id'],
                     aggfunc='count'
                             ).reset_index()
#only sept has the matching cb307... dataset key
#April to May, had 65 routes with the 43a1e... dataset key. but 66 routes in Sept.
#What is the difference between these months? What routes are in each of these months

## Questions to answer regarding `sac_trip_desc`
1. Why does September have 66 route_ids?
    <br>a. What are the unique routes in each month?
2. Why does September have a different dataset key, where as the other months have a differnt key?
    <br>a. do april, may, july, aug have the same routes and therefore the same dataset key?
3. 

In [ ]:
#funcion that prints the unique route IDs for each month in the helpers.import_scheduled_trips dataset

def route(month):
    array = sac_trip_desc[sac_trip_desc.month==month].route_id.unique()
    series = pd.Series(array).sort_values()
    
    return series

In [ ]:
#attempt to compare the routes in each month from helpers.import_scheduled_trips dataframe
#which routes exisit / not exist compared to other months. 
april = route('april')
may = route('may')
july = route('july')
aug = route('aug')
sep = route('sep')

### Need to find a way to compare each of these months to see which routes appear/not appear in each month
Then i can move on with next step

## Attempt at joining sac_m4m9 and sac_trip_desc
1. first update the col names of sac_m4m9 to match sac_trip_desc (8 = aug, 9 = sep)

In [ ]:
#test function that creates a new col in sac_m4m9 and tags a new value
def month_def(x):
    if x.month == 4:
        return 'april'
    elif x.month == 5:
        return 'may'
    elif x.month == 6:
        return 'june'
    elif x.month == 7:
        return 'july'
    elif x.month == 8:
        return 'aug'
    elif x.month == 9:
        return 'sep'

In [ ]:
sac_m4m9['month_name'] = sac_m4m9.apply(month_def,axis=1)

In [ ]:
sac_m4m9.month_name.value_counts()

In [ ]:
#test join, just to see what happenes
join = pd.merge(sac_m4m9, sac_trip_desc, on='schedule_gtfs_dataset_key', how ='left')
list(join.columns)

# HALL OF SHAME

---


---


In [ ]:
#DO NOT RUN



# grabbing additional dates
# april_date = rt_dates.DATES["apr2023"]
# may_date = rt_dates.DATES["may2023"]
# june_date = rt_dates.DATES["jun2023"]
# july_date = rt_dates.DATES["jul2023"]
# aug_date = rt_dates.DATES["aug2023"]
# sep_date = rt_dates.DATES["sep2023"]

In [ ]:
#DO NOT RUN
# list of all the months
# months_list = [sep_date, aug_date, april_date, may_date, june_date, july_date]

In [ ]:
#DO NOT RUN
# test loop of import_scheduled_trips, iterated through months_list

# empty list
# month_trips = {}

# for months in months_list:
#     df = helpers.import_scheduled_trips(
#         months,
#         columns=[
#             "feed_key",
#             "name",
#             "gtfs_dataset_key",
#             "route_id",
#             "route_short_name",
#             "route_long_name",
#             "route_desc",
#         ],
#         filters=[[("name", "==", "Sacramento Schedule")]],
#         get_pandas=True,
#     )
#     month_trips[months] = df

In [ ]:
#  month_trips.keys()

In [ ]:
# april = month_trips['2023-04-12'].assign(month='april')
# may = month_trips['2023-05-17'].assign(month='may')
# june = month_trips['2023-06-14'].assign(month='june')
# july = month_trips['2023-07-12'].assign(month='july')
# aug = month_trips['2023-08-15'].assign(month='aug')
# sep = month_trips['2023-09-13'].assign(month='sep')

In [ ]:
# use this pseudo code to concat all the different months data
# pd.concat can stack DFs on top of others.
# pd.concat([df1, df2], axis=0)

In [ ]:
#DO NOT RUN
#test to see if i can use a dictionary in concat
# all_trips = pd.concat([april,may,june,july,aug,sep],axis=0)

In [ ]:
#DO NOT RUN
#saving dataframe to gcs
#TEST CODE aprl_sept_2023_routes.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/route_identification_2023_m04_m09.parquet')

# all_trips.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_trips_route_identification_2023_m04_m09.parquet')

In [ ]:
#peaking into df to make sure everything looks good

#shape shows 11,927 rows and 8 columns
#display(df.shape)

#type shows data is in df
#display(type(df))

#columns return all the columns we listed in the function
#display(list(df.columns))

#value_counts confirm df only has rows from 2023 March to May
#display(df.value_counts(subset=['year','month']))

### via Tiffany, make df to GPD
* <b>COMPLETE</b> Query data from warehouse
* then use this snippet from `make_routes_gdf` from `_shared_utils/shared_utils/geography_utils.py`.
    * `ddf["geometry"] = ddf.pt_array.apply(make_linestring)`
<br>  
* <b>COMPLETE</b> then save out as geo parquet to the `gtfs_schedule` folder in GCS (so versioning and history stays) using 
    * `utils.geoparquet_gcs_export(vp_gdf, SEGMENT_GCS, f"vp_{analysis_date}")`

In [ ]:
#test of make_routes_gdf. DO NOT RUN

# aprl_sept_2023_routes = geography_utils.make_routes_gdf(df, "EPSG:4326")

In [ ]:
#display(type(aprl_sept_2023_routes))
#list(aprl_sept_2023_routes.columns)
#display(aprl_sept_2023_routes.geometry.name)

---

In [ ]:
#creating sub-df for 'Sacramento Schedule'. DO NOT RUN

# sac = aprl_sept_2023_routes[aprl_sept_2023_routes['name'] == 'Sacramento Schedule']

In [ ]:
# type(sac)

In [ ]:
#writing sac filtered gdf to gcs as geoparquet. DO NOT RUN AGAIN

# utils.geoparquet_gcs_export(sac, "gs://calitp-analytics-data/data-analyses/gtfs_schedule/", "sac_route_identification_2023_m04_m09_geo")

In [ ]:
#Now we can read in the parquet file from gcs without having to remake everything again.

sac_gdf = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_route_identification_2023_m04_m09_geo.parquet')

In [ ]:
type(sac_gdf)

In [ ]:
display(sac.shape)
display(sac.head())

In [ ]:
sac_gdf.plot()

In [ ]:
Attempt at creating sub df of each routes to plot

In [ ]:
sac_routes = sac_gdf['route_id'].unique()


In [ ]:
#empty dictionary
sac_sub_route_ids = {}

#each element in sac_routes will be called route.
#for each route in sac_routes, query each row related to that route.(where ever you use the variable route, go 1-by-1 the differnet
#then, create a dataframe for each route and place it into the dictionary sub_dataframes
for route in sac_routes:
    sub_df = sac[sac['route_id'] == route]
    sac_sub_route_ids[route] = sub_df

In [ ]:
len(sac_sub_route_ids)

In [ ]:
#testing dictionary with route 23 and 88
sac_sub_route_ids['023']

In [ ]:
#function to plot routes and to check for any variation in route geometry
def route_checker(route_id):
    display(sac_sub_route_ids[route_id].plot())
    display(f'geometry checker...{sac_sub_route_ids[route_id].geometry.value_counts()}')
    

In [ ]:
route_checker('088')

At this point determined we need more data that what `fct_monthly_routes` has. See approach 2

## Next Steps

for every `name` and `route_id` in routes, need to see if each row is the same or not. Need to identify any variation in the routes. 



---

Trying to use a loop that will create a df for every route_id in sac_routes. but would need to do this for every `name` in the `fct_monthly_routes` df eventually?


---

In [ ]:
#list of unique route names from initial df
route_names = df['name'].unique()


In [ ]:
#new loop that creates a dictionary of each unique schedule name with all its routes.
sub_route_name = {}

for name in route_names:
    sub_df2 = df[df['name'] == name]
    sub_route_name[name] = sub_df2
    

In [ ]:
#test to see if new dictionary works
sub_route_name['Auburn Schedule']

## Now I have `sac_sub_route_ids` and `sub_route_name` dictionaries

Examples of some noteable routes with slight variations over time.

In [ ]:
#General observations for Sacramento Schedule: 
#shape_id changes every month. 
#pt_array changes every month, however, did get a warning upon initial query of data so may need to review query to account for geodata
#Month 4 has the point geom data

display(sac_sub_route_ids['088'])
display(sac_sub_route_ids['023'])
display(sac_sub_route_ids['105'])
display(sac_sub_route_ids['F20'])
display(sac_sub_route_ids['215'])

In [ ]:
#test to see other route names
#other route names have more point geometry than Sacramento.
display(sub_route_name['Santa Cruz Schedule'])
display(sub_route_name['Merced Schedule'])
display(sub_route_name['San Diego Schedule'])
display(sub_route_name['Roseville Schedule'])

In [ ]:
import importlib

importlib.reload(segment_speed_utils)
from segment_speed_utils.project_vars import SCHED_GCS, analysis_date